# Querying

We assume that you have [generated a SDK](generation.html) for the `Windmill` model and have a client ready to go.

Querying is a different way of retrieving data than doing `.list()`, `.retrieve()`, `search()` in that it supports retrieving nested data structures. 
SDKs generated by `pygen` have two ways of querying which we will refer to as Python querying and GraphQL qurying. In this section, we will go through both of them,
and list limitations as well as when to use one over the other. 



In [1]:
import warnings
warnings.filterwarnings('ignore')
# This is just to enable improting the generated SDK from the examples folder in the pygen repository
import sys
from tests.constants import REPO_ROOT
sys.path.append(str(REPO_ROOT / "examples" ))

In [2]:
from windmill import WindmillClient

In [3]:
client = WindmillClient.from_toml("config.toml")

## Comparing: Python and GraphQL based Querying

<img src="images/windmill_type.png" width="400">


### Python Based Querying

**Advantages**

* Discovarability through the IDE by using available docstrings.
* The returning data classes have required/optional set as in the data model, which are useful for static type checking and IDE auto complete.

**Limitations**

* You can only query along one chain of edges. For example, if we start from `Windmill` above we can either go to blades or metmast, not both.
* You can only include direct relations at the end of the chain of edges. In the example above, we cannot get a windmill with rotor object, and blades with sensor positions.

### GraphQL Based Querying

**Advantages**

* Flexiblity in querying. You can retrieve anything you can write up as a graphql query. 

**Limitations**

* Difficult to write the querying, typically you would use the CDF UI to create the query.
* The returning data class has all properties as optional and is thus less structured.

### Summary

GraphQL based querying is more flexible, but gives you less structure on the returning data classes. In addition, the query will have to be created outside of your IDE which requires context switching.

## Python Based Querying

In Python based Querying, we chain together python methods call do filter (select) what instances we want to get. We finish with a `.query()` which executes the query.

### Querying along Edges

`pygen` supports quering nodes via edges. This includes handeling circular dependencies. 

For example, if we want to list the all the blades with sensor positoins we can do the followind query:

In [5]:
blades = client.blade(limit=5).sensor_positions(limit=-1).query()

In [6]:
blades[0].model_dump()

{'space': 'windmill-instances',
 'external_id': 'blade:1',
 'data_record': {'version': 1,
  'last_updated_time': datetime.datetime(2023, 12, 25, 7, 47, 50, 40000, tzinfo=TzInfo(UTC)),
  'created_time': datetime.datetime(2023, 12, 25, 7, 47, 50, 40000, tzinfo=TzInfo(UTC)),
  'deleted_time': None},
 'node_type': None,
 'is_damaged': False,
 'name': 'A',
 'sensor_positions': [{'space': 'windmill-instances',
   'external_id': 'sensorposition:1',
   'data_record': {'version': 1,
    'last_updated_time': datetime.datetime(2023, 12, 25, 7, 47, 50, 40000, tzinfo=TzInfo(UTC)),
    'created_time': datetime.datetime(2023, 12, 25, 7, 47, 50, 40000, tzinfo=TzInfo(UTC)),
    'deleted_time': None},
   'node_type': None,
   'edgewise_bend_mom_crosstalk_corrected': None,
   'edgewise_bend_mom_offset': None,
   'edgewise_bend_mom_offset_crosstalk_corrected': None,
   'edgewisewise_bend_mom': None,
   'flapwise_bend_mom': 'V52-WindTurbine.MxA6',
   'flapwise_bend_mom_crosstalk_corrected': None,
   'flapw

**Call Explaination**
The call `client.blade(limit=5).sensor_positions(limit=-1).query()` requires some extra explaination. 

1. The first call, `.blade(limit=5)`, filters which blades that we want.
2. In the second call, `.sensor_positions(limit=-1)`, we specify that we want all sensor position edges connected to the blades we found in the first call.
3. The last call `.query()` tells `pygen` to execute the query.


We can build arbitrary long queries this way and we always finish with `.query()` to execute the query. For example, we can take all windmills at the 'Hornsea 1' windfarm and include all blades and sensor position connected to these blades.

In [7]:
windmills_at_hornsea_with_blades_and_sensors = client.windmill(windfarm="Hornsea 1").blades(limit=-1).sensor_positions(limit=-1).query()

In [8]:
windmills_at_hornsea_with_blades_and_sensors

,space,external_id,blades,capacity,metmast,nacelle,name,rotor,windfarm,node_type,data_record
0,windmill-instances,hornsea_1_mill_3,"[{'space': 'windmill-instances', 'external_id'...",7.0,None,nacellewrite:1,hornsea_1_mill_3,rotorwrite:1,Hornsea 1,None,"{'version': 2, 'last_updated_time': 2024-02-10..."
1,windmill-instances,hornsea_1_mill_2,"[{'space': 'windmill-instances', 'external_id'...",7.0,None,nacellewrite:2,hornsea_1_mill_2,rotorwrite:2,Hornsea 1,None,"{'version': 2, 'last_updated_time': 2024-02-10..."
2,windmill-instances,hornsea_1_mill_1,"[{'space': 'windmill-instances', 'external_id'...",7.0,None,nacellewrite:3,hornsea_1_mill_1,rotorwrite:3,Hornsea 1,None,"{'version': 2, 'last_updated_time': 2024-02-10..."
3,windmill-instances,hornsea_1_mill_4,"[{'space': 'windmill-instances', 'external_id'...",7.0,None,nacellewrite:4,hornsea_1_mill_4,rotorwrite:4,Hornsea 1,None,"{'version': 2, 'last_updated_time': 2024-02-10..."
4,windmill-instances,hornsea_1_mill_5,"[{'space': 'windmill-instances', 'external_id'...",7.0,None,nacellewrite:5,hornsea_1_mill_5,rotorwrite:5,Hornsea 1,None,"{'version': 2, 'last_updated_time': 2024-02-10..."


In [9]:
for windmill in windmills_at_hornsea_with_blades_and_sensors:
    print(f"{windmill.name} has {len(windmill.blades)} blades")
    for blade in windmill.blades:
        print(f"{blade.external_id} is connected to {windmill.name} and has sensors: {', '.join([sensor.external_id for sensor in blade.sensor_positions])}")
    print("\n")

hornsea_1_mill_3 has 6 blades
blade:1 is connected to hornsea_1_mill_3 and has sensors: sensorposition:1, sensorposition:2, sensorposition:3, sensorposition:4, sensorposition:5, sensorposition:6
blade:2 is connected to hornsea_1_mill_3 and has sensors: sensorposition:7, sensorposition:8, sensorposition:9, sensorposition:10, sensorposition:11, sensorposition:12
blade:3 is connected to hornsea_1_mill_3 and has sensors: sensorposition:13, sensorposition:14, sensorposition:15, sensorposition:16, sensorposition:17, sensorposition:18
bladewrite:1 is connected to hornsea_1_mill_3 and has sensors: sensorpositionwrite:1, sensorpositionwrite:2, sensorpositionwrite:3, sensorpositionwrite:4, sensorpositionwrite:5, sensorpositionwrite:6
bladewrite:2 is connected to hornsea_1_mill_3 and has sensors: sensorpositionwrite:7, sensorpositionwrite:8, sensorpositionwrite:9, sensorpositionwrite:10, sensorpositionwrite:11, sensorpositionwrite:12
bladewrite:3 is connected to hornsea_1_mill_3 and has sensors: 

We can dig into the nested structure of one of them by dumping it do a dictinary.

In [10]:
windmills_at_hornsea_with_blades_and_sensors[1].model_dump()

{'space': 'windmill-instances',
 'external_id': 'hornsea_1_mill_2',
 'data_record': {'version': 2,
  'last_updated_time': datetime.datetime(2024, 2, 10, 9, 30, 16, 893000, tzinfo=TzInfo(UTC)),
  'created_time': datetime.datetime(2023, 12, 25, 7, 47, 50, 40000, tzinfo=TzInfo(UTC)),
  'deleted_time': None},
 'node_type': None,
 'blades': [{'space': 'windmill-instances',
   'external_id': 'blade:4',
   'data_record': {'version': 1,
    'last_updated_time': datetime.datetime(2023, 12, 25, 7, 47, 50, 40000, tzinfo=TzInfo(UTC)),
    'created_time': datetime.datetime(2023, 12, 25, 7, 47, 50, 40000, tzinfo=TzInfo(UTC)),
    'deleted_time': None},
   'node_type': None,
   'is_damaged': True,
   'name': 'B',
   'sensor_positions': [{'space': 'windmill-instances',
     'external_id': 'sensorposition:19',
     'data_record': {'version': 1,
      'last_updated_time': datetime.datetime(2023, 12, 25, 7, 47, 50, 40000, tzinfo=TzInfo(UTC)),
      'created_time': datetime.datetime(2023, 12, 25, 7, 47, 5

Notice that when we query we can easily get very large structures. So we should always use query with filtering options to only query the specific part of the data you have an interesst. 

### Querying with Direct Relations

We can include the direct relations by specifing them on the `.query()` call

In [11]:
windmills = client.windmill(limit=2).query(retrieve_nacelle=True, retrieve_rotor=True)

In [12]:
windmills[0].nacelle

,value
space,windmill-instances
external_id,nacellewrite:1
data_record,"{'version': 2, 'last_updated_time': 2024-02-10..."
node_type,None
acc_from_back_side_x,V52-WindTurbine.Acc1N
acc_from_back_side_y,V52-WindTurbine.Acc2N
acc_from_back_side_z,V52-WindTurbine.Acc3N
gearbox,gearboxwrite:1
generator,generatorwrite:1
high_speed_shaft,highspeedshaftwrite:1


## GraphQL based Querying

When querying with GraphQL we must include `__typename` of the top level items as this is used by `pygen` to understand how to pase the object.

The querying method is available on the top level client as this is not particular to any of the data types in your data model

In [14]:
my_query = """{
  listWindmill(first:1){
    items{
      __typename
      name
      capacity
      nacelle{
        externalId
      }
      rotor{
        externalId
      }
      blades{
        items{
          name
          is_damaged
        }
      }
  }
 }
}"""

In [15]:
result = client.graphql_query(my_query)

In [16]:
result

,space,external_id,blades,capacity,metmast,nacelle,name,rotor,windfarm,data_record
0,None,None,"[{'space': None, 'external_id': None, 'data_re...",7.0,None,"{'space': None, 'external_id': 'nacellewrite:1...",hornsea_1_mill_3,"{'space': None, 'external_id': 'rotorwrite:1',...",None,None


In [19]:
result[0].model_dump(exclude_none=True)

{'blades': [{'is_damaged': False, 'name': 'A'},
  {'is_damaged': False, 'name': 'B'},
  {'is_damaged': False, 'name': 'C'},
  {'is_damaged': False, 'name': 'A'},
  {'is_damaged': False, 'name': 'B'},
  {'is_damaged': False, 'name': 'C'}],
 'capacity': 7.0,
 'nacelle': {'external_id': 'nacellewrite:1'},
 'name': 'hornsea_1_mill_3',
 'rotor': {'external_id': 'rotorwrite:1'}}

In [20]:
turbine = result[0]

In [21]:
turbine.nacelle.external_id

'nacellewrite:1'

## Pitfalls

If you forget to include `__typename` on the top level item `pygen` will raise a `Runtime` error

In [22]:
my_invalid_query = """
{
  listWindmill(first:1){
    items{
      name
  }
 }
}
"""

In [23]:
client.graphql_query(my_invalid_query)

RuntimeError: Missing '__typename' in GraphQL response. Cannot determine the type of the response.

## Data Classes

When you call `.list()`, `.retrieve()` and `.search(), `pygen` returns ther read format of data classes. This read format matches the type/view required/optional properties. 

When you do the `graphql_query` above all properties are optional as `pygen` cannot know which objects you included in your query, thus `pygen` uses a special GraphQL format of the 
data class it is returning

In [25]:
my_query = """{
  listWindmill(first:1){
    items{
      name
      __typename
  }
 }
}"""

In [26]:
result = client.graphql_query(my_query)

In [27]:
type(result)

windmill.data_classes._core.GraphQLList

In [28]:
type(result[0])

windmill.data_classes._windmill.WindmillGraphQL

This data class can be converted to a regular write or read format by calling the `as_write` and `as_read` call.

**Warning** If you have not included all required properties in the your GraphQL query, `pygen` will raise an `ValueError` when you do this call.

In [29]:
result[0].as_read()

ValueError: This object cannot be converted to a read format because it lacks a data record.

In [33]:
my_query = """{
  listWindmill(first:1){
    items{
      name
      space
      externalId
      createdTime
      lastUpdatedTime
      __typename
  }
 }
}"""

In [34]:
result = client.graphql_query(my_query)

In [37]:
windmill_read = result[0].as_read()
windmill_read

,value
space,windmill-instances
external_id,hornsea_1_mill_3
data_record,"{'version': 0, 'last_updated_time': 2024-02-10..."
node_type,None
blades,[]
capacity,None
metmast,[]
nacelle,None
name,hornsea_1_mill_3
rotor,None


In [38]:
type(windmill_read)

windmill.data_classes._windmill.Windmill

In [41]:
windmill_write = result[0].as_write()
windmill_write

,value
space,windmill-instances
external_id,hornsea_1_mill_3
data_record,{'existing_version': 0}
node_type,None
blades,[]
capacity,None
metmast,[]
nacelle,None
name,hornsea_1_mill_3
rotor,None


In [42]:
type(windmill_write)

windmill.data_classes._windmill.WindmillWrite

Next section: [Creating and Deleting](creating_deleting.html)